In [30]:
import pandas as pd
import numpy as np
import random
import math
from typing import Dict, List, Tuple

import plotly.express as px
pd.options.plotting.backend = "plotly"

In [31]:
# Market behavior simulation functions
def short_sin(day:int, cycle:int):
    value = 1.5 + (0.5 * math.sin((day - 7.5) / (cycle / (2*math.pi))))
    return value


def short_cos(day:int, cycle:int):
    value = 1.5 + (0.5 * math.cos((day + cycle) / (cycle / (2*math.pi))))
    return value


def long_sin(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi)))) * (10 - (day / cycle)) / 10
    #value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


def long_cos(day:int, cycle:int, offset:float, amplitude:float):
    value = amplitude * (1 + (0.5 * math.cos(((day + 2 * cycle * offset) % (2 * cycle)) / (cycle / math.pi))) * (10 - (day / (2 * cycle))) / 10 )
    #value = 1 + (0.5 * math.cos(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


# Reward rate framework
def rr_framework(supply:int):
    if supply > 10000000000000:
      return 0.0000625
    elif supply > 1000000000000:
      return 0.000125
    elif supply > 100000000000:
      return 0.00025
    elif supply > 10000000000:
      return 0.0005
    elif supply > 1000000000:
      return 0.001
    elif supply > 1000000000:
      return 0.001
    elif supply > 100000000:
      return 0.002
    else:
      return 0.004

In [36]:
class ModelParams():
    def __init__(self, horizon:int, ask_capacity:float, bid_capacity:float, target_ma:float, lower_band:float, upper_band:float, min_premium_target:int, max_outflow_rate:float, supply_amplitude:int, reserve_change_speed:float, max_liq_ratio:float, cycle_reweights:float, release_capture:float, demand_factor:float, supply_factor:float, initial_supply:float, initial_reserves:float, initial_liq_usd:float, arb_factor:float, initial_price:float, initial_target:float, target_price_function:str, short_cycle:int, long_cycle:int, long_sin_offset:float, long_cos_offset:float):
        self.horizon = horizon
        self.cycle_reweights = cycle_reweights
        self.reserve_change_speed = reserve_change_speed
        self.short_cycle = short_cycle
        self.long_cycle = long_cycle
        self.long_sin_offset = long_sin_offset
        self.long_cos_offset = long_cos_offset
        self.supply_amplitude = supply_amplitude

        self.max_liq_ratio = max_liq_ratio
        self.min_premium_target = min_premium_target
        self.release_capture = release_capture
        self.max_outflow_rate = max_outflow_rate
        self.demand_factor = demand_factor
        self.supply_factor = supply_factor

        self.target_ma = target_ma
        self.lower_band = lower_band
        self.upper_band = upper_band
        self.bid_capacity = bid_capacity
        self.ask_capacity = ask_capacity


        self.initial_supply = initial_supply
        self.initial_reserves = initial_reserves
        self.initial_liq_usd = initial_liq_usd
        self.initial_price = initial_price
        self.initial_target = initial_target
        self.target_price_function = target_price_function
        self.arb_factor = arb_factor


class Day():
    def __init__(self, params:ModelParams, prev_arbs:Dict[int, Tuple[float, float]], prev_lags=Dict[int, Tuple[int, Dict[int, float]]], prev_day=None):

        if prev_day is None:
            self.day = 1
            self.supply = params.initial_supply
            self.reward_rate = rr_framework(self.supply)
            self.price = params.initial_price
            self.natural_target = params.initial_target
            self.real_target = params.initial_target
            self.lower_target = params.initial_target * (1 - params.lower_band)
            self.upper_target = params.initial_target * (1 + params.upper_band)
            self.bid_capacity = 0
            self.ask_capacity = 0
            self.bid_used = 0
            self.ask_used = 0
            self.liq_usd = params.initial_liq_usd
            self.liq_ohm = self.liq_usd / self.price
            self.k = (self.liq_usd ** 2) / self.price
            self.reserves_in = 0
            self.reserves_out = 0
            self.release_capture = 0
            self.ohm_traded = 0
            self.cum_ohm_purchased = 0
            self.reserves = params.initial_reserves
            self.prev_reserves = params.initial_reserves
            self.market_demand = params.demand_factor
            self.market_supply = params.supply_factor
            self.arb_factor = params.arb_factor
            self.arb_demand = 0
            self.arb_supply = 0
            self.unwind_demand = 0
            self.unwind_supply = 0
            self.net_flow = random.uniform(self.liq_usd * self.market_supply, self.liq_usd * self.market_demand)
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)

        else:
            self.day = prev_day.day + 1
            self.reward_rate = rr_framework(prev_day.supply)
            self.supply = prev_day.supply * (1 + self.reward_rate)
            self.natural_target = calc_natural_target(params=params, prev_day=prev_day)
            self.real_target = calc_price_target(params=params, prev_day=prev_day, prev_lags=prev_lags)
            self.lower_target = self.real_target * (1 - params.lower_band)
            self.upper_target = self.real_target * (1 + params.upper_band)

            
            # Market dynamics
            self.net_flow = random.uniform(prev_day.mcap * prev_day.total_supply, prev_day.mcap * prev_day.total_demand) + prev_day.release_capture
            #self.net_flow = random.uniform(prev_day.liq_usd * prev_day.total_supply, prev_day.liq_usd * prev_day.total_demand) + prev_day.release_capture

            self.market_demand = params.demand_factor * short_sin(self.day, params.short_cycle) * long_sin(self.day, params.long_cycle, params.long_sin_offset)
            self.market_supply = params.supply_factor * short_cos(self.day, params.short_cycle) * long_cos(self.day, params.long_cycle, params.long_cos_offset, params.supply_amplitude)
            
            #if prev_day.price < prev_day.lower_target:
            #    self.arb_factor = (prev_day.lower_target / prev_day.price) ** ((self.day % (params.short_cycle)) / (params.short_cycle / 2)) - 1
            #elif prev_day.price > prev_day.upper_target:
            #    self.arb_factor = (prev_day.upper_target / prev_day.price) ** ((self.day % (params.short_cycle)) / (params.short_cycle / 2)) - 1
            #else:
            #    self.arb_factor = 0
            #self.arb_demand = params.demand_factor * self.arb_factor
            #self.arb_supply = params.supply_factor * (1 / (1 + self.arb_factor) - 1)

            #if self.day >= params.short_cycle:
            #        unwind_date = self.day - (2 * self.day % (params.short_cycle)) - 1
            #        self.unwind_demand = prev_arbs[unwind_date][1] * (params.release_capture - 1)
            #        self.unwind_supply = prev_arbs[unwind_date][0] * (1 - params.release_capture)
            #else:
            #    self.unwind_demand = 0
            #    self.unwind_supply = 0
            #prev_arbs[self.day] = (self.arb_demand, self.arb_supply)
            
            
            # Guidance capacity
            if prev_day.price > prev_day.lower_target:
                self.bid_capacity = params.bid_capacity * prev_day.reserves
            else:
                self.bid_capacity = prev_day.bid_capacity - prev_day.reserves_out

            if prev_day.price < prev_day.upper_target:
                self.ask_capacity = params.ask_capacity * prev_day.floating_supply * prev_day.fmcap_treasury_ratio / params.min_premium_target
            else:
                self.ask_capacity = prev_day.ask_capacity - prev_day.ohm_traded


            # AMM k
            if prev_day.fmcap_treasury_ratio > params.min_premium_target:
                self.k = prev_day.k * (1 + self.reward_rate)**2
            else:
                self.k = prev_day.k #* ((1 + self.reward_rate)**2) * (params.min_premium_target / (prev_day.fmcap_treasury_ratio))


            
            # Reserve capture
            if prev_day.price > prev_day.lower_target or prev_day.price < prev_day.upper_target:
                if prev_day.reserves * (1 - params.max_liq_ratio) < prev_day.liq_usd * params.max_liq_ratio:
                    if prev_day.price < prev_day.real_target:
                        self.reserves_in = (prev_day.liq_usd * params.max_liq_ratio - prev_day.reserves * (1 - params.max_liq_ratio)) / ((params.reserve_change_speed / 2) * params.short_cycle)
                    else:
                        self.reserves_in = (prev_day.liq_usd * params.max_liq_ratio - prev_day.reserves * (1 - params.max_liq_ratio)) / ((params.reserve_change_speed / 0.5) * params.short_cycle)
                #elif prev_day.liq_usd * (1 - params.max_liq_ratio) < prev_day.reserves * params.max_liq_ratio:
                else:
                    if prev_day.price < prev_day.real_target:
                        self.reserves_in = -2 * (prev_day.reserves * params.max_liq_ratio - prev_day.liq_usd * (1 - params.max_liq_ratio)) / ((params.reserve_change_speed / 2) * params.short_cycle)
                    else:
                        self.reserves_in = -2 * (prev_day.reserves * params.max_liq_ratio - prev_day.liq_usd * (1 - params.max_liq_ratio)) / ((params.reserve_change_speed / 0.5) * params.short_cycle)
            else:
                self.reserves_in = 0
            
            #if self.reserves_in > (-1) * prev_day.reserves * params.max_outflow_rate:
            #    self.reserves_in = (-1) * prev_day.reserves * params.max_outflow_rate


            # Liquidity USD
            if prev_day.price < prev_day.lower_target:
                self.liq_usd = (prev_day.lower_target * self.k)**(1/2)
            elif prev_day.price > prev_day.upper_target:
                self.liq_usd = (prev_day.upper_target * self.k)**(1/2)
            else:
                self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_in  # same as prev_day.liq_usd + self.net_flow_and_bond

            # Reserve release
            self.reserves_out = self.liq_usd - prev_day.liq_usd - self.net_flow

            # if reserve release is > capacity left, we need to re-do the previous calcs
            if prev_day.price < prev_day.lower_target and self.reserves_out > self.bid_capacity:
                self.reserves_out = self.bid_capacity
                self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_out

            # if reserve capture is > capacity left, we need to re-do the previous calcs
            if prev_day.price > prev_day.upper_target and (-1) * self.reserves_out > self.ask_capacity:
                self.reserves_out = (-1) * self.ask_capacity
                self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_out

            # Remaining Liquidity and Reserve variables
            self.reserves = prev_day.reserves - self.reserves_out
            self.prev_reserves = prev_day.reserves
            self.liq_ohm = self.k / self.liq_usd
            self.price = self.liq_usd / self.liq_ohm
            self.ohm_traded = (-2) * self.reserves_out / (self.price + prev_day.price)
            self.cum_ohm_purchased = prev_day.cum_ohm_purchased - self.ohm_traded

            #still necessary?
            if self.day % params.short_cycle == 0:
                self.release_capture = (-1) * self.reserves_out * params.release_capture
            else:
                self.release_capture = 0


            if prev_day.price < prev_day.lower_target:
                self.bid_used = prev_day.bid_used + self.reserves_out
            else:
                self.bid_used = 0
            if prev_day.price > prev_day.upper_target:
                self.ask_used = prev_day.ask_used + self.ohm_traded
            else:
                self.ask_used = 0
        

        self.net_flow_and_bond = self.net_flow - self.reserves_in
        self.treasury = self.liq_usd + self.reserves
        self.mcap = self.supply * self.price
        self.floating_supply = self.supply - self.liq_ohm - self.cum_ohm_purchased
        self.floating_mcap = self.floating_supply * self.price
        self.liq_ratio = self.liq_usd / self.reserves
        self.reserves_ratio = self.reserves / self.liq_usd
        self.fmcap_treasury_ratio = self.floating_mcap / self.treasury
        self.liq_fmcap_ratio = self.liq_usd / self.floating_mcap
        #self.net_arb = self.arb_demand + self.arb_supply + self.unwind_demand + self.unwind_supply
        #self.total_demand = self.market_demand + self.arb_demand + self.unwind_demand
        #self.total_supply = self.market_supply + self.arb_demand + self.unwind_supply
        self.total_demand = self.market_demand
        self.total_supply = self.market_supply
        self.total_net = self.total_demand + self.total_supply


        # Only for reporting purposes (to check calculations)
        prev_lags['price'][1][self.day] = self.price
        prev_lags['target'][1][self.day] = self.real_target
        prev_lags['natural'][1][self.day] = self.natural_target


In [43]:
# Target price controller
def calc_price_target(params:ModelParams, prev_day:Day, prev_lags:Dict[int, Tuple[int, Dict[int, float]]]):
    if params.target_price_function == 'avg_lags':
        if prev_day.day % (params.short_cycle) == 0:
            s = 0
            lag_keys = set(prev_lags.keys()) - set(['price', 'target', 'natural', 'avg'])
            for key in lag_keys:
                days = prev_lags[key][1].keys()
                s += prev_lags[key][1][max(days)]
            avg_lag = s / len(lag_keys)
            return (prev_day.natural_target + avg_lag) / 2
        else:
            return prev_day.real_target

    elif params.target_price_function == 'price_cycle_avg':
        if prev_day.day % (params.short_cycle) == 0:
            s = 0
            days = len(prev_lags['price'][1]) - 1
            days_reweight = int(params.short_cycle)
            if days > params.short_cycle:
                for i in range(days - days_reweight, days):
                    s += prev_lags['price'][1][i]
                return s / days_reweight
            else:
                 return prev_day.real_target
        else:
            return prev_day.real_target

    elif params.target_price_function == 'price_moving_avg':
        s = 0
        days = len(prev_lags['price'][1]) - 1
        days_ma = int(params.target_ma)
        if days > params.target_ma:
            for i in range(days - days_ma, days):
                print(days, days_ma, i, s)
                s += prev_lags['price'][1][i]
            return s / days_ma
        else:
            return prev_day.real_target


def calc_natural_target(params:ModelParams, prev_day:Day):
    if prev_day.day % (params.short_cycle) == 0:
      if prev_day.day % (params.short_cycle * 4) == 0:
        return ((prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves) + prev_day.real_target) / 2
      else:
        return prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves
    else:
        return prev_day.natural_target


def calc_lag(day:int, params:ModelParams, prev_lags:Dict[int, Tuple[int, Dict[int, float]]], num_days:int=3):
    for key, values in prev_lags.items():
        if key not in ('price', 'target', 'natural', 'avg'):
          lag_days = values[0]
          if day > lag_days:
              if key == 'lag1':
                  if day > params.short_cycle:
                      s = prev_lags['price'][1][day-1]
                      for i in range(1, num_days):
                          s += prev_lags['price'][1][day - (i * lag_days)]
                      prev_lags[key][1][day] = s / num_days
                  else:
                    prev_lags[key][1][day] = values[1][day - 1]
              else:
                prev_lags[key][1][day] = prev_lags['lag1'][1][day - values[0]]
          else:
            prev_lags[key][1][day] = values[1][day - 1]


### __SET SCENARIO PARAMETERS__

In [46]:
# Simulate scenario
params = ModelParams(horizon = 360  # simulation timespan.
    ,short_cycle = 90  # short market cycle duration.
    ,cycle_reweights = 1  # reweights per short market cycle.
    ,long_cycle = 730  # long market cycle duration.
    ,long_sin_offset = 2  # demand function offset.
    ,long_cos_offset = 0  # supply function offset.
    ,supply_amplitude=0.8  # supply function amplitude.
    
    # Initial Parameters
    ,initial_supply = 25000000, initial_reserves = 280000000, initial_liq_usd = 70000000, initial_price = 25, initial_target = 25, target_price_function = 'price_moving_avg'

    ,demand_factor = 0.008  # % of OHM supply expected to drive market demand.
    ,supply_factor = -0.008  # % of OHM supply expected to drive market sell preasure.
    ,arb_factor = 0  # initial arb factor
    ,release_capture = 0.333  # % of reweight taken immediately by the market.

    ,max_liq_ratio = 0.5  # liquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = 3  # minimum premium to keep adding liquidity as supply grows.
    ,max_outflow_rate = 0.0033 # max % of reservesUSD that can be released on a single day
    ,reserve_change_speed=0.5  # directly related to the speed at which reserves are released/captured by the treasury. The higher the slower.
    ,ask_capacity=0.05  # % of floating supply that the treasury can deploy when price is trading above the upper target.
    ,bid_capacity=0.1  # % of the reserves that the treasury can deploy when price is trading below the lower target.
    ,target_ma=90  # length of the price target moving average (in days).
    ,lower_band=0.25  # determines lower price target at x% below the target price.
    ,upper_band=0.25  # determines upper price target at x% above the target price.
)

lags = {
    'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'natural': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target}),
    'lag1': (30, {1: params.initial_target}), 'lag2': (90, {1: params.initial_target}), 'lag3': (180, {1: params.initial_target}), 'lag4': (360, {1: params.initial_target})
}

arbs = {}

simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}

for i in range (2, params.horizon):
    simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])
    calc_lag(day=i, params=params, prev_lags=lags)

91 90 1 0
91 90 2 25
91 90 3 54.74001610893579
91 90 4 83.63706242835505
91 90 5 113.13977974796765
91 90 6 135.9267530876098
91 90 7 161.95370910913078
91 90 8 187.76899962053275
91 90 9 209.1601424699524
91 90 10 239.00305937000485
91 90 11 276.1946531143488
91 90 12 311.9765862846779
91 90 13 358.77882878008705
91 90 14 419.02800135585255
91 90 15 502.88344350247513
91 90 16 626.6556274869874
91 90 17 731.3480926336101
91 90 18 857.7950611177062
91 90 19 1028.4422049281918
91 90 20 1312.4987638862506
91 90 21 1474.513914211857
91 90 22 1558.2507805490081
91 90 23 1677.8383464529613
91 90 24 1774.1187222157666
91 90 25 1870.69929596146
91 90 26 1990.784549486403
91 90 27 2180.3604185189615
91 90 28 2499.2062685002297
91 90 29 2621.9174763702895
91 90 30 2754.27382700361
91 90 31 2830.6372496861177
91 90 32 2892.6789684883006
91 90 33 2950.6715916401367
91 90 34 3019.4265024515626
91 90 35 3083.1378659132074
91 90 36 3171.8721461978093
91 90 37 3288.2064495536565
91 90 38 3368.0765016

ZeroDivisionError: float division by zero

### __PLOT RESULTS__

In [28]:
# Protocol variables
df = pd.DataFrame(columns = ['NetFlow', 'Price', 'RealTarget', 'LowerTarget', 'UpperTarget', 'NaturalTarget', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio', 'ReserveRatio', 'LiqFloatingMCRatio']) 
for day, data in simulation.items():
    df.loc[day] = [data.net_flow, data.price, data.real_target, data.lower_target, data.upper_target, data.natural_target, data.liq_usd, data.liq_ohm, data.k, data.reserves, 100*data.reserves/data.prev_reserves, data.reserves_in, data.reserves_out, data.treasury, data.supply, data.mcap, data.floating_supply, data.floating_mcap, data.liq_ratio, data.reserves_ratio, data.liq_fmcap_ratio]

# Guidance variables
guidance_df = pd.DataFrame(columns = ['BidCapacity', 'BidUsed', 'AskCapacity', 'AskUsed', 'ReservesIN', 'ReservesOUT']) 
for day, data in simulation.items():
    guidance_df.loc[day] = [data.bid_capacity, data.bid_used, data.ask_capacity, data.ask_used, data.reserves_in, data.reserves_out]

# Market dynamics variables
#market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb'])
market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'NetTotal'])
for day, data in simulation.items():
    #market_df.loc[day] = [data.market_demand, data.market_supply, data.arb_factor, data.arb_demand, data.arb_supply, data.unwind_demand, data.unwind_supply, data.total_demand, data.total_supply, data.total_net, data.net_arb]
    market_df.loc[day] = [data.market_demand, data.market_supply, data.total_net]


In [29]:
# Plot multivariable charts

fig = df[['NetFlow', 'LiqUSD', 'Reserves', 'Treasury']].plot()
fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
fig.show()


fig1 = df[['MCap','FloatingMCap', 'LiqUSD']].plot()
fig1.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig1.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
fig1.show()


fig2 = df[['Price','RealTarget', 'LowerTarget', 'UpperTarget', 'ReserveChange']].plot()
fig2.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig2.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
fig2.show()


fig3 = market_df[['MarketDemand', 'MarketSupply']].plot()
fig3.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, params.horizon, params.short_cycle):
    fig3.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
fig3.show()


#fig4 = market_df[['ArbSupply', 'UnwindDemand']].plot()
#fig4.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
#for i in range (params.short_cycle, params.horizon, params.short_cycle):
#    fig4.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
#fig4.show()

fig5 = guidance_df[['BidCapacity', 'BidUsed', 'ReservesOUT']].plot()
fig5.show()
fig6 = guidance_df[['AskCapacity', 'AskUsed', 'ReservesOUT']].plot()
fig6.show()


In [ ]:
# Plot batch1
for col in df.columns:
    if col in ('NetFlow', 'LiqRatio', 'LiqFloatingMCRatio'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        if col == 'LiqRatio':
            fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="grey")
        if col == 'LiqFloatingMCRatio':
            fig.add_hline(y=0.2, line_width=1, line_dash="dash", line_color="grey")
        fig.show()

In [ ]:
# Plot batch2
for col in df.columns:
    if col in ('Reserves', 'LiqUSD', 'LiqOHM', 'ReservesIN', 'ReservesOUT', 'Treasury'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        fig.show()

In [ ]:
# Plot batch3
for col in df.columns:
    if col in ('Supply', 'FloatingSupply', 'MCap', 'FloatingMCap'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        fig.show()

In [ ]:
# Plot batch4
for col in ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb']:
    fig = market_df.plot(x=df.index, y=market_df[col])
    fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    for i in range (params.short_cycle, params.horizon, params.short_cycle):
        fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
    fig.show()


In [ ]:
# Market cycles (sin and cos waves)
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 2*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset)]

df2.plot(y=df2.columns)

In [ ]:
# All variables related to price controller
price_target_df = pd.DataFrame(columns = ['Price', 'Target', 'Natural', 'Avg', 'Lag1', 'Lag2', 'Lag3', 'Lag4']) 
for key in lags.keys():
    for day, p in lags[key][1].items():
      price_target_df.at[day, key.capitalize()] = p

price_target_df.plot()